# 모델링 구현

## word2vec을 활용한 모델 구현

- 각 단어에 대해 word2vec으로 벡터화
- 전처리된 데이터를 불러온 후 각 단어들의 리스트로 나눠야 함

In [15]:
import pandas as pd
import numpy as np

In [8]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

sentences = []
for review in reviews:
    sentences.append(review.split())

- 전처리한 데이터의 경우 리뷰가 하나의 문자열로 구성되어 있음
- word2vec를 사용하기 위해서는 입력값을 단어로 구분된 리스트로 만들어야함

### word2vec 벡터화

In [9]:
# word2vec 모델의 하이퍼파라미터 설정
# 학습 시 필요한 하이퍼파라미터
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

- num_features
    - 워드 벡터 특징값 수
    - 각 단어에 대해 임베딩된 벡터의 차원을 정한다.

- min_word_count
    - 단어에 대한 최소 빈도 수
    - 모델에 의미 있는 단어를 가지고 학습하기 위해 적은 빈도 수의 단어들은 학습하지 않는다.
    
- num_workers
    - 프로세스 개수
    - 모델 학습 시 학습을 위한 프로세스 개수를 지정한다.

- context
    - 컨텍스트 윈도우 크기
    - word2vec을 수행하기 위한 컨텍스트 윈도우 크기를 지정한다.

- downsampling
    - 다운 샘플링 비율
    - word2vec 학습을 수행할 때 빠른 학습을 위해 정답 단어 라벨에 대한 다운샘플링 비율을 지정한다. 보통 0.001이 좋은 성능을 낸다고 한다.

In [10]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

- 로깅을 할 때 format을 위와 같이 지정하고 로그 수준은 INFO에 맞추면 word2vec의 학습 과정에서 로그 메세지를 양식에 맞게 INFO 수준으로 보여준다

- word2vec 모듈에 있는 Word2Vec 객체를 생성해서 실행
- 학습하고 생성된 객체는 model 변수에 할당

In [11]:
from gensim.models import word2vec

print("Training model.....")
model = word2vec.Word2Vec(
    sentences,
    workers = num_workers,
    size = num_features,
    min_count = min_word_count,
    window = context,
    sample = downsampling)

2020-02-14 22:39:54,355 : INFO : collecting all words and their counts
2020-02-14 22:39:54,359 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
Training model.....
2020-02-14 22:39:55,422 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2020-02-14 22:39:56,482 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2020-02-14 22:39:57,189 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2020-02-14 22:39:57,191 : INFO : Loading a fresh vocabulary
2020-02-14 22:39:58,140 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2020-02-14 22:39:58,144 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2020-02-14 22:39:58,307 : INFO : deleting the raw counts dictionary of 74065 items
2020-02-14 22:39:58,314 : INFO : sample=0.001 downsamples 30 most-common words
2020-

- 모델의 하이퍼파라미터를 설정한 내용을 모델 이름에 담는다면 나중에 참고하기에 좋을 것이다.
- 모델을 저장하면 Word2Vec.load()를 통해 모델을 다시 사용할 수 있다.

In [12]:
model_name = "300features_10minwords_10context"
model.save(model_name)

2020-02-14 22:45:45,905 : INFO : saving Word2Vec object under 300features_10minwords_10context, separately None
2020-02-14 22:45:45,907 : INFO : not storing attribute vectors_norm
2020-02-14 22:45:45,911 : INFO : not storing attribute cum_table
2020-02-14 22:45:46,672 : INFO : saved 300features_10minwords_10context


### 선형 회귀 모델 학습

- 입력값 만드는 방법
    - 문장에 있는 모든 단어의 벡터값에 대해 평균을 내서 리뷰 하나당 하나의 벡터로 만드는 방법

In [16]:
# 하나의 리뷰에 대해 전체 단어의 평균값을 계산하는 함수
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype=np.float32)

    num_words = 0

    # 어휘사전 준비
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])

    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

- words
    - 단어의 모음인 하나의 리뷰가 들어간다.

- model
    - word2vec 모델을 넣는 곳이며, 우리가 학습한 word2vec 모델이 들어간다.

- num_features
    - word2vec으로 임베딩할때 정했던 벡터의 차원 수를 뜻한다.